# 1. Seleção

Nesta primeira etapa, definimos a base do nosso projeto.

+ O que fizemos: Extraímos o Adult Data Set (Censo) do repositório da UCI.

+ A nossa escolha: Separamos os dados estruturais (as features, como idade, educação e horas de trabalho) da nossa variável-alvo (target), que é prever se a renda da pessoa ultrapassa os 50 mil dólares anuais.

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

In [ ]:
adult = fetch_ucirepo(id=2)

In [ ]:
df = adult.data.features
df.head()

In [ ]:
target = adult.data.targets
target.head()

In [ ]:
df["income"] = target

# 2. Pré-Processamento

Aqui, investigamos a qualidade dos dados e lidamos com valores ausentes (como os campos em branco ou preenchidos com "?").

+ O que fizemos: Comparamos empiricamente várias estratégias de limpeza para ver qual ajudava mais o modelo.

+ A nossa escolha: Descobrimos que não remover linhas com dados faltantes era a melhor opção. Ao apagar dados com "?" ou nulos, nossa acurácia caiu para 86,56%. A escolha final foi tratar esses dados ausentes preenchendo-os com a moda (valor mais frequente) e removendo as colunas "education-num", "fnlwgt devido, a feature "education-num" ser apenas um id de "education" feature, falando sobre "fnlwgt" ajuste na distribuição seguindo uma tranformação logmod.

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Metodos</th>
      <th>acurácia</th>
      <th>Modelo</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Sem alterações no preprocessamento </td>
      <td> 87,69% </td>
      <td rowspan="9" style="vertical-align: middle;">XGBoost</td>
    </tr>
    <tr>
      <td>Removendo linhas com Valores nulos</td>
      <td> 87,49% </td>
    </tr>
    <tr>
      <td>Substituindo Valores nulos pela moda</td>
      <td> 87,78% </td>
    </tr>
    <tr>
      <td>Substituir nulos pela moda e dropar duplicados</td>
      <td> 87,54% </td>
    </tr>
    <tr>
      <td>Remover linhas com nulos e duplicados</td>
      <td>87,19%</td>
    </tr>
    <tr>
      <td>Remover linhas com valores "?"</td>
      <td>86,56%</td>
    </tr>
    <tr>
      <td>Substituir valores "?" pela moda</td>
      <td>87,59%</td>
    </tr>
    <tr>
      <td>Substituir pela moda valores nulos e "?"</td>
      <td>87,63%</td>
    </tr>
    <tr style="font-weight: bold;">
      <td>Valores nulos pela moda e drop de 2 colunas e ajuste na distribuição de fnlwgt</td>
      <td>87,94%</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
print(f"Shape inicial: {df.shape}")
df.head()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

corr = df.select_dtypes(include=['number']).corr().abs()

plt.figure(figsize=(6,4))
sns.heatmap(corr, cmap='coolwarm')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = df.select_dtypes(include=['int', 'float'])

for col in num_cols:
    plt.figure(figsize=(10,4))

    plt.subplot(1,2,1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution: {col}')

    plt.subplot(1,2,2)
    sns.boxenplot(df[col])
    plt.title(f'Boxplot: {col}')

    plt.show()

In [ ]:
import numpy as np

df['fnlwgt_logmod'] = np.sign(df['fnlwgt']) * np.log1p(np.abs(df['fnlwgt']))

In [ ]:
print("Valores ausentes por coluna:")
print(df.isna().sum())
print(f"\nTotal de linhas com valores ausentes: {df.isna().any(axis=1).sum()}")

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:

nan_columns = df.columns[df.isna().any()].tolist()
print(f"Colunas com valores ausentes: {nan_columns}")
for column in nan_columns:
    print(f"Preenchendo valores ausentes na coluna '{column}' com a moda.")
    df[column] = df[column].fillna(df[column].mode()[0])
print("\nVerificação:")
print(df.isna().sum())

In [ ]:
for col in df.columns:
    n_unique = df[col].nunique()
    print(f"{col} ({n_unique} valores únicos):")
    print(df[col].unique())
    print()

In [ ]:
print("Valores originais de 'income':")
print(df["income"].unique())

df["income"] = df["income"].str.replace(".", "").str.strip()

print("\nValores após limpeza:")
print(df["income"].unique())

In [ ]:
print("Distribuição de 'occupation':")
print(df["occupation"].value_counts())
print(f"\nRegistros com '?': {(df['occupation'] == '?').sum()}")

In [ ]:
print("Distribuição de 'native-country':")
print(df["native-country"].value_counts())
print(f"\nRegistros com '?': {(df['native-country'] == '?').sum()}")

In [ ]:
print("Colunas antes da remoção:")
print(df.columns.tolist())

df = df.drop(columns=["education-num", "fnlwgt"])

print("\nColunas após remoção:")
print(df.columns.tolist())
print(f"\nShape final: {df.shape}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = ['fnlwgt_logmod']

for col in num_cols:
    plt.figure(figsize=(10,4))

    plt.subplot(1,2,1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution: {col}')

    plt.subplot(1,2,2)
    sns.boxenplot(df[col])
    plt.title(f'Boxplot: {col}')

    plt.show()

# 3. Transformação

Modelos matemáticos não leem textos e são sensíveis a escalas numéricas diferentes. Nesta fase, traduzimos os dados para o modelo.

+ O que fizemos: Transformamos variáveis categóricas (como "Private" e "Local-gov") em números através do One-Hot Encoding e ajustamos escalas numéricas.

+ A nossa escolha: Testamos tanto a Padronização quanto a Normalização dos dados. Como ambas empataram em 87,94% de acurácia, comprovamos que o modelo escolhido (baseado em árvores) é robusto e invariante à escala.

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Metodos</th>
      <th>acurácia</th>
      <th>Modelo</th>
    </tr>
  </thead>
  <tbody>
  <tr>
      <td><b>Sem métodos de scaler</b></td>
      <td><b>87,94%</b></td>
      <td rowspan="3" style="vertical-align: middle;">XGBoost</td>
    </tr>
    <tr>
      <td><b>Padronizando os dados</b></td>
      <td><b>87,94%</b></td>
    </tr>
    <tr>
      <td><b>Normalizando os dados</b></td>
      <td><b>87,94%</b></td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
print(f"Shape do dataset: {df.shape}")
df.head()

In [ ]:
y = (df["income"] == ">50K").astype(int)
X = df.drop(columns=["income"])

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print("\nDistribuição da variável target:")
print(y.value_counts())
print("\nProporção:")
print(y.value_counts(normalize=True))

In [ ]:
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

print(f"Variáveis numéricas ({len(numerical_cols)}):")
print(numerical_cols)
print(f"\nVariáveis categóricas ({len(categorical_cols)}):")
print(categorical_cols)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de y_test: {y_test.shape}")

print("\nDistribuição em treino:")
print(y_train.value_counts(normalize=True))
print("\nDistribuição em teste:")
print(y_test.value_counts(normalize=True))

In [ ]:
print("Aplicando One-Hot Encoding...")

X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=False)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=False)

print("\nShape após encoding:")
print(f"X_train_encoded: {X_train_encoded.shape}")
print(f"X_test_encoded: {X_test_encoded.shape}")

In [ ]:
print("Alinhando colunas entre treino e teste...")

X_train_encoded, X_test_encoded = X_train_encoded.align(
    X_test_encoded,
    join="left",
    axis=1,
    fill_value=0,
)

print("\nShape após alinhamento:")
print(f"X_train_encoded: {X_train_encoded.shape}")
print(f"X_test_encoded: {X_test_encoded.shape}")
print(f"\nColunas idênticas: {X_train_encoded.columns.equals(X_test_encoded.columns)}")

In [ ]:
print("Colunas numéricas a serem normalizadas:")
print(numerical_cols)

existing_num_cols = [col for col in numerical_cols if col in X_train_encoded.columns]
print("\nColunas numéricas presentes:")
print(existing_num_cols)

In [ ]:
standard = False
scaler = StandardScaler() if standard else MinMaxScaler()

X_train_encoded[existing_num_cols] = scaler.fit_transform(X_train_encoded[existing_num_cols])
X_test_encoded[existing_num_cols] = scaler.transform(X_test_encoded[existing_num_cols])

print("\nEstatísticas das variáveis numéricas no treino:")
print(X_train_encoded[existing_num_cols].describe())

# 4. Data Mining

Esta é a fase de aprendizado, onde aplicamos o algoritmo principal para extrair os padrões preditivos da base.

+ O que fizemos: Definimos o XGBoost como nosso modelo principal devido à sua alta performance com dados tabulares.

+ A nossa escolha: Para tentar melhorar o modelo, aplicamos técnicas de busca de hiperparâmetros (como o Grid Search e Random Search). Curiosamente, observamos que os parâmetros padrão do XGBoost já eram extremamente otimizados para essa base (87,94%), tendo um desempenho até levemente superior à busca exaustiva que fizemos (87,88%), o que nos ensinou sobre o risco de overfitting na otimização.

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Metodos</th>
      <th>acurácia</th>
      <th>Modelo</th>
    </tr>
  </thead>
  <tbody>
  <tr>
      <td><b>Sem métodos de hiperparametros</b></td>
      <td><b>87,94%</b></td>
      <td rowspan="4" style="vertical-align: middle;">XGBoost</td>
    </tr>
    <tr>
      <td>Grid Search</td>
      <td>87,88%</td>
    </tr>
    <tr>
      <td>Random Search</td>
      <td>87,88%</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
models = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=1000, random_state=42)
    },

    "Random Forest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1),
    },

    "XGBoost": {
        "model": XGBClassifier(random_state=42, eval_metric="logloss", n_jobs=-1),
    },
}

In [ ]:
results = {}

for name, mp in models.items():
    model = mp['model']
    model.fit(X_train_encoded, y_train.values.ravel())
    test_score = model.score(X_test_encoded, y_test)

    results[name] = {
        "model": model,
        "params": model.get_params(),
        "test_score": test_score
    }

In [ ]:
comparison = pd.DataFrame({
    "model": list(results.keys()),
    "test_score": [results[m]["test_score"] for m in results],
}).sort_values("test_score", ascending=False)

print(comparison.to_string(index=False))

In [ ]:
best_model_name = comparison.iloc[0]["model"]
best_model = results[best_model_name]['model']

print(f"Best Model: {best_model_name}")
print(f"Test Accuracy: {results[best_model_name]['test_score']:.4f}")

# 5. Avaliação

A fase final serviu para comprovar matematicamente que nosso modelo funciona e pode generalizar o conhecimento.

+ O que fizemos e concluímos: Avaliamos todas as modificações anteriores mantendo a métrica de Acurácia como nossa bússola. Conseguimos provar que, com o tratamento correto de dados categóricos e respeitando a natureza do algoritmo (sem deletar dados cegamente), estabilizamos a capacidade preditiva do modelo em quase 88% de acerto no mundo real.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score

In [ ]:
y_pred = best_model.predict(X_test_encoded)
print(classification_report(y_test, y_pred, target_names=["<=50K", ">50K"]))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["<=50K", ">50K"],
    yticklabels=["<=50K", ">50K"],
)
plt.ylabel("Valor Real")
plt.xlabel("Valor Predito")
plt.title("Matriz de Confusão")
plt.show()

print(f"Verdadeiros Negativos (<=50K corretos): {cm[0,0]}")
print(f"Falsos Positivos (previu >50K, era <=50K): {cm[0,1]}")
print(f"Falsos Negativos (previu <=50K, era >50K): {cm[1,0]}")
print(f"Verdadeiros Positivos (>50K corretos): {cm[1,1]}")